In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
from common_import import *

In [10]:
from mpl_toolkits.mplot3d.axes3d import Axes3D

In [11]:
fig = plt.figure(0)

type(fig.add_axes([0, 0, 1, 1], projection='3d')) == Axes3D


True

In [2]:
data_type_list = ['source_list', 'cam_param', 
                  'world_3d', 'cam_3d', 'img_2d', 
                  #'img_3d', 'scale_factor', 'img_25d', 
                  #'cam_3d_canonical_same_z', 'img_2d_canonical_same_z', 
                  #'cam_3d_canonical_same_dist', 'img_2d_canonical_same_dist',
                  'cam_3d_canonical_fixed_dist_5', 'img_2d_canonical_fixed_dist_5']
overwrite_list = []
h36m = load_data_dict('h36m', data_type_list, overwrite_list, verbose=False)
fit3d = load_data_dict('fit3d', data_type_list, overwrite_list, verbose=False)
_3dhp = load_data_dict('3dhp', data_type_list, overwrite_list, verbose=False)

In [5]:
list(h36m.keys()) == data_type_list

True

In [7]:
for dataset_name in ['h36m', 'fit3d', '3dhp']:
    if dataset_name == 'h36m': data = h36m.copy()
    elif dataset_name == 'fit3d': data = fit3d.copy()
    elif dataset_name == '3dhp': data = _3dhp.copy()
    img_2d_canonicals_fixed_dist_5 = []
    cam_params = []
    for source in data['source_list']:
        if dataset_name == '3dhp':
            if 'TS' not in source: continue
        subject, cam_id, action = split_source_name(source, dataset_name)
        img_2d_canonicals_fixed_dist_5 += list(data[f'img_2d_canonical_fixed_dist_5'][subject][action][cam_id]) 
        cam_param = data['cam_param'][subject][action][cam_id]
        num_frames = cam_param['num_frames']
        cam_params += [cam_param] * num_frames
    img_2d_canonicals_fixed_dist_5 = np.array(img_2d_canonicals_fixed_dist_5)
    cam_params = np.array(cam_params)

    W_array = np.array([cam_param['W'] for cam_param in cam_params])
    H_array = np.array([cam_param['H'] for cam_param in cam_params])
    fx_array = np.array([cam_param['intrinsic'][0][0] for cam_param in cam_params])
    fy_array = np.array([cam_param['intrinsic'][1][1] for cam_param in cam_params])
    cx_array = np.array([cam_param['intrinsic'][0][2] for cam_param in cam_params])
    cy_array = np.array([cam_param['intrinsic'][1][2] for cam_param in cam_params])
    img_2d_canonicals_fixed_dist_5_norm = img_2d_canonicals_fixed_dist_5.copy()
    img_2d_canonicals_fixed_dist_5_norm *= 2
    img_2d_canonicals_fixed_dist_5_norm -= np.concatenate([W_array[:, None, None], H_array[:, None, None]], axis=2)
    img_2d_canonicals_fixed_dist_5_norm /= W_array[:, None, None]

    x_max = np.max(img_2d_canonicals_fixed_dist_5_norm[:, :, 0], axis=1) # max x value for each frame
    x_min = np.min(img_2d_canonicals_fixed_dist_5_norm[:, :, 0], axis=1) # min x value for each frame
    y_max = np.max(img_2d_canonicals_fixed_dist_5_norm[:, :, 1], axis=1) # max y value for each frame
    y_min = np.min(img_2d_canonicals_fixed_dist_5_norm[:, :, 1], axis=1) # min y value for each frame

    avg_diff_x = (x_max - x_min).mean()
    avg_diff_y = (y_max - y_min).mean()
    avg_fx = fx_array.mean()
    avg_fy = fy_array.mean()
    print(f"{dataset_name}: avg_diff_x: {avg_diff_x:.2f}, avg_diff_y: {avg_diff_y:.2f}, avg_fx: {avg_fx:.2f}, avg_fy: {avg_fy:.2f}")


h36m: avg_diff_x: 0.26, avg_diff_y: 0.62, avg_fx: 1147.34, avg_fy: 1146.24
fit3d: avg_diff_x: 0.29, avg_diff_y: 0.65, avg_fx: 1062.13, avg_fy: 1056.99
3dhp: avg_diff_x: 0.21, avg_diff_y: 0.41, avg_fx: 1506.39, avg_fy: 1506.14
